# 딥러닝 중간시험 2021년 2학기

**이름**: 임형택

**학번**: 2017250039

**제출내역**: "파일 -> 다운로드 -> .ipynb 다운로드" 선택 후 다운로드된 주피터 노트북을 사이버캠퍼스 과제란에 있는 "중간고사 프로그래밍 과제"에서 제출할 것.

## 문제

"케라스와 함께하는 딥러닝 첫 프로젝트"란 주제로 작은 프로젝트를 구현하는 전 과정을 설명하는 블로그를 작성하라.

제한조건은 다음과 같다.

- 주제: [캐글의 타이타닉 생존자 예측](https://www.kaggle.com/c/titanic)

- 사용 모델: 케라스의 `Sequential` 모델과 `Dense` 층만 활용할 것
- 포함 내용: [5장 머신러닝의 핵심 이슈](https://codingalzi.github.io/dlp/notebooks/dlp05_fundamentals_of_ml.html) 전반

시험 평가 지표는 다음과 같다(각 20점).

- 이해도
- 친절성
- 완전성
- 다양성
- 효율성

### 문제 풀이

여기서부터 시작하세요.

**타이타닉 데이터 가져오기**

In [1]:
!git clone https://github.com/hyoungteak/Assignment.git

Cloning into 'Assignment'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 216 (delta 26), reused 40 (delta 7), pack-reused 115
Receiving objects: 100% (216/216), 12.75 MiB | 36.07 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [2]:
import pandas as pd

train = pd.read_csv('/content/Assignment/data/train.csv')
test = pd.read_csv('/content/Assignment/data/test.csv')
gender_submission = pd.read_csv('/content/Assignment/data/gender_submission.csv')

In [3]:
# 타이타닉 데이터 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


우선 수정하거나 부족한 부분을 처리해보자.

### **성별(Sex)**
성별(Sex) 부분이 `male` or `female`로 저장되어 있으니, 수치형 데이터로 변환한다.

In [5]:
train['Sex'] = train['Sex'].astype('category').cat.codes
test['Sex'] = test['Sex'].astype('category').cat.codes

### **Embarked**
또한 `Embarked` 항목의 NaN 값을 가장 많은 데이터가 있는 'S'로 채운다.

In [6]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [7]:
train['Embarked'].fillna('S', inplace=True)

# test 셋은 문제 X
train['Embarked'] = train['Embarked'].astype('category').cat.codes
test['Embarked'] = test['Embarked'].astype('category').cat.codes

### **나이(Age)**
사실 타이타닉 경진 대회에서 제일 중요한 것이 나이, 성별 방의 위치인데, 깊게 파고 들자면 구명보트를 태웠을 때, 여자와 어린이 먼저 태웠기 때문이다.

그래서 `Age`에 대한 빈 값이 상당히 많이 존재하는데, `Median` 값으로 채웠다.

결과 예측 값을 보고 `Age`에 대한 데이터 설정을 수정하여, 성능을 올릴 수 있을 것이다.

In [8]:
train['Age'].median()

28.0

In [9]:
median = train["Age"].median()
train["Age"].fillna(median, inplace=True)
test["Age"].fillna(median, inplace=True)

그리고 임의의 5개 그룹으로 묶어주자

In [10]:
train['AgeBand'] = pd.cut(train['Age'], 5)
# 임의로 5개 그룹을 지정
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.344168
2,"(32.252, 48.168]",0.404255
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


### **Cabin**
카빈은 객실을 뜻하는데, 알파벳 + 숫자 형식으로 되어있다.

다 보기는 무리니까 알파벳만 따서 보도록 하자.

개인적으로 방 번호에 숫자가 1이면 개인, 여러 명이면 단체로 놓는 feature를 추가하면 어떨까 싶다.

우선은 그냥 nan값이니까 N으로 채워서 진행해보자.

In [11]:
# 앞의 알파벳만 가져오기
train['Cabin'].str[:1].value_counts()

C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [12]:
train['Cabin'].fillna('N', inplace=True)
test['Cabin'].fillna('N', inplace=True)

이것도 수치형으로 바꾸기 위해서 값을 지정해서 넣어주자.

In [13]:
cabin_num = {
    'N' : 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8
}

train['Cabin'] = train['Cabin'].map(cabin_num)
test['Cabin'] = test['Cabin'].map(cabin_num)

In [14]:
# 타이타닉 데이터 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    int8    
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        691 non-null    float64 
 11  Embarked     891 non-null    int8    
 12  AgeBand      891 non-null    category
dtypes: category(1), float64(3), int64(5), int8(2), object(2)
memory usage: 72.6+ KB


### **학습 시작**

우선 `tensorflow`를 임포트한다.

In [22]:
import tensorflow as tf
import numpy as np

In [19]:
train.drop(['Cabin'], inplace=True, axis=1)

In [23]:
train_y = np.ravel(train.Survived)

In [28]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(16, activation='relu', input_shape=(3,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
model.fit(train, train_y, epochs=50, batch_size=1, verbose=1)

ValueError: ignored